# Insurance Claim Analysis Scenario with Semantic Kernel and Azure OpenAI Assistants API

## Context

In May 2024, Hurricane Alicia strikes the Gulf Coast near Houston, Texas, causing extensive property damage. Fabrikam Insurance, a leading commercial insurer, now faces numerous complex claims. One such claim is from Contoso Electronics Inc. for their retail location #TX-230 in Houston.

The claim involves severe roof damage, flooding of inventory storage, compromised electrical systems, and potential business interruption. The insurer’s underwriting and claims team uses Semantic Kernel agents to orchestrate a multi-step analysis:

- **Review** the claim details from the adjuster’s report.
- **Verify** policy coverage, deductibles, and limits.
- **Assess** the severity of damage, detect any fraud indicators, and recommend next steps.

## Multi-Agent Process Using Semantic Kernel's Agent Framework

We orchestrate a multi-agent process:

1. **Claims Analyst**: Reviews claim documents, identifies key details, and notes any missing information.
2. **Policy Verifier**: Checks coverage limits, applies correct deductibles, and identifies endorsements or exclusions.
3. **Risk Assessor**: Evaluates the severity of the incident against the weather data, assesses mitigation efforts, and provides forward-looking recommendations.

The `AgentGroupChat` coordinates these agents until a final recommendation is formed.

In [ ]:
!pip install semantic-kernel==1.17.0 python-dotenv aiofiles nest_asyncio azure-search-documents pyperclip


## Imports & Configuration

In [15]:
import asyncio
import os
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import logging

from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

import nest_asyncio
nest_asyncio.apply()

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-01-preview")

if not all([AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT]):
    raise ValueError("Missing Azure OpenAI configuration.")

logging.getLogger().setLevel(logging.WARNING)


## Create Kernel and Documents

In [29]:
def create_kernel():
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel

def save_sample_documents():
    # Claim Report (Houston scenario)
    claim_report = f"""
INSURANCE CLAIM REPORT
----------------------
Claim ID: HURTX-2024-0456
Date Filed: {datetime.now().strftime('%B %d, %Y')}
Claimant: Contoso Electronics Inc.
Store Location: Contoso Electronics #TX-230
Address: 1125 Main Street, Houston, TX 77002
Contact: Gabriel Allen (Store Manager)
Contact Phone: (713) 555-4422
Contact Email: gabriel.allen@contoso.com

DAMAGE ASSESSMENT
-----------------
Incident Type: Hurricane Alicia
Date of Incident: May 15, 2024
Time of Incident: ~02:30 AM CST

DAMAGE DETAILS
--------------
- Severe roof damage and water intrusion on the south side
- Flooded inventory storage (electronics submerged)
- Electrical panels shorted due to water ingress
- Potential structural compromise in south wing support beams

Preliminary Damage Estimates:
- Inventory Loss: $380,000
- Structural Damage: $200,000
- Equipment Replacement: $160,000
- Business Interruption: TBD (further accounting needed)

Mitigation Steps Taken:
1. Secure site and prevent unauthorized entry
2. Engage emergency restoration team for water extraction
3. Move salvageable inventory to upper floors
4. Temporary roof patch installed

Photos and videos available. Awaiting professional structural engineer’s report.

FILED BY
--------
Adjuster: Lauren Chen
Fabrikam Insurance Adjuster ID: TX-ADJ-3112
Agency: Gulf Coast Claims Solutions
Initial On-Site Visit: {(datetime.now() + timedelta(days=1)).strftime('%B %d, %Y')}
""".strip()

    # Policy Details (Fabrikam Insurance)
    policy_details = f"""
FABRIKAM INSURANCE COMMERCIAL POLICY
-------------------------------------
Policy Number: CP-TX-2024087
Policyholder: Contoso Electronics Inc.
Policy Period: January 1, 2024 - December 31, 2024
Policy Type: Comprehensive Commercial Property Insurance

COVERAGE & LIMITS
-----------------
Total Insured Value: $2,000,000
Hurricane/Wind Coverage: Included
Flood Coverage: Included (Deductible: $50,000)
Business Interruption: Included (sub-limit $500,000)
Replacement Cost: Included

Sublimits:
- Building Structure: Up to $1,000,000
- Business Property (Incl. Inventory): Up to $600,000
- Electronic Equipment: Up to $200,000

DEDUCTIBLES
-----------
Hurricane/Wind Deductible: 5% of total insured value ($100,000)
Flood Deductible: $50,000
Standard Deductible: $25,000

ENDORSEMENTS & NOTES
--------------------
- Hurricane Preparedness Credit applied
- Extended Replacement Cost Endorsement active
- Business Interruption Rider active

UNDERWRITING
------------
Underwriter: Fabrikam Insurance Underwriting Dept.
Last Risk Assessment: April 5, 2024 (No major hazards noted at that time)
Annual Premium: $42,000 paid in full on January 5, 2024
    """.strip()

    # Weather Report (Hurricane Alicia in Houston)
    weather_report = f"""
NOAA SEVERE WEATHER REPORT
--------------------------
Report Date: {datetime.now().strftime('%B %d, %Y')}
Event: Hurricane Alicia (2024 Season)

STORM DETAILS
-------------
Category: 3
Max Sustained Winds: ~120 mph
Min Central Pressure: ~950 mb

HOUSTON AREA IMPACT
-------------------
Landfall: Near Galveston, TX
Time: May 15, 2024, ~02:10 AM CST
Duration of Severe Conditions: 6-8 hours over Houston

METEOROLOGY
-----------
Winds: Sustained 100-120 mph, Gusts 130+ mph
Rainfall: 10-14 inches total, peak 3 in/hr
Storm Surge: 8-10 feet along coastal areas

DAMAGE POTENTIAL
----------------
Infrastructure Risk: Very High
Property Damage Likelihood: High
Business Disruption Probability: Significant

Issued By: Dr. Marisol Reyes
National Hurricane Center - Houston Office
""".strip()

    base_directory = Path.cwd() / "insurance_docs"
    base_directory.mkdir(exist_ok=True)

    documents = {
        "claim_report.txt": claim_report,
        "policy_details.txt": policy_details,
        "weather_report.txt": weather_report,
    }

    for fname, content in documents.items():
        (base_directory / fname).write_text(content, encoding="utf-8")

    return [str(base_directory / f) for f in documents.keys()]

document_files = save_sample_documents()


## Creating Agents

We will have three agents:

1. **Claims Analyst (with File Search)**: Reviews claim, policy, and weather documents.
2. **Policy Verifier**: Checks coverage and compliance.
3. **Risk Assessor**: Evaluates risks and provides recommendations.

In [50]:
async def create_agents(kernel: Kernel, document_files: list[str]):
    claims_analyst = await AzureAssistantAgent.create(
        kernel=kernel,
        deployment_name=AZURE_OPENAI_DEPLOYMENT,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        name="Claims_Analyst",
        instructions="""You are an experienced Insurance Claims Analyst evaluating claim HURTX-2024-0456 for hurricane damage.

YOUR PROCESS:
1. First Response: Initial analysis only
- Review claim details and identify missing information
- DO NOT make recommendations yet

2. Final Response: Only after Policy_Verifier and Risk_Assessor input
- Consider their input for coverage and risk assessment
- Make final recommendation
- Structure in JSON format:
{
    "final_recommendation": "approve"|"deny"|"partial_approve",
    "rationale": [...],
    "approved_amount": number,
    "deductions": {...},
    "conditions": [...]
}

KEY REQUIREMENTS:
- Wait for both other agents before final recommendation
- Verify damage aligns with weather data
- Flag any inconsistencies
- Consider policy limits and deductibles""",
        enable_file_search=True,
        vector_store_filenames=document_files
    )

    policy_verifier = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Policy_Verifier",
        instructions="""You are a Policy Verification Specialist examining claim HURTX-2024-0456.

RESPOND IMMEDIATELY AFTER Claims_Analyst's initial review with:
{
    "coverage_verification": {
        "hurricane_coverage": {...},
        "flood_coverage": {...},
        "business_interruption": {...}
    },
    "applicable_deductibles": [...],
    "available_limits": {...},
    "coverage_gaps": [...],
    "policy_compliance": "compliant"|"non_compliant",
    "special_conditions": [...]
}

KEY FOCUS:
- Apply correct hurricane vs flood deductibles
- Verify all claimed damages are covered
- Calculate remaining policy limits
- Identify any coverage exclusions""",
    )

    risk_assessor = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Risk_Assessor",
        instructions="""You are a Risk Assessment Specialist analyzing claim HURTX-2024-0456.

RESPOND AFTER Policy_Verifier with:
{
    "risk_analysis": {
        "weather_correlation": {...},
        "mitigation_effectiveness": {...},
        "future_risks": [...]
    },
    "damage_validation": {
        "consistent_with_storm": boolean,
        "evidence_gaps": [...],
        "suspicious_patterns": [...]
    },
    "mitigation_assessment": {...},
    "recommendations": [...],
    "risk_score": number
}

KEY REQUIREMENTS:
- Verify damage timing matches storm data
- Assess adequacy of emergency measures
- Identify potential future risks
- Flag any suspicious patterns""",
    )

    return claims_analyst, policy_verifier, risk_assessor

## Setting Up Group Chat
We’ll define a selection strategy to determine which agent responds next and a termination strategy to know when the conversation is complete. For demo purposes, we might simplify the logic or leave it as-is.

In [47]:
async def setup_group_chat(kernel: Kernel, claims_analyst, policy_verifier, risk_assessor):
    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt="""You determine which agent speaks next in the claims analysis conversation.
        
Previous messages: {{{$history}}}

Strict speaking order:
1. Claims_Analyst first (initial review)
2. Then Policy_Verifier 
3. Then Risk_Assessor
4. Finally Claims_Analyst again (final decision)

Rules:
- Output ONLY the exact name: Claims_Analyst, Policy_Verifier, or Risk_Assessor
- No explanations or other text
- Must follow the order above exactly
- If the last message was from Claims_Analyst and didn't mention a final recommendation, output "Policy_Verifier"
- If the last message was from Policy_Verifier, output "Risk_Assessor"
- If the last message was from Risk_Assessor, output "Claims_Analyst"
- If no messages yet, output "Claims_Analyst"
""")

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""You check if the claims analysis conversation is truly complete.

Previous messages: {{{$history}}}

Requirements for COMPLETE status:
1. Claims_Analyst gave initial review
2. THEN Policy_Verifier analyzed coverage
3. THEN Risk_Assessor evaluated risks
4. FINALLY Claims_Analyst gave final recommendation

Output only one word:
- "COMPLETE" if ALL steps happened IN ORDER
- "INCOMPLETE" if any steps are missing or out of order

Special rules:
- The conversation can't be complete if less than 4 messages
- Must have a final structured recommendation from Claims_Analyst as last message
- If Claims_Analyst only gave initial review, it's INCOMPLETE
""")

    group_chat = AgentGroupChat(
        agents=[claims_analyst, policy_verifier, risk_assessor],
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=kernel,
            result_parser=lambda r: str(r.value[0]) if r.value else "Claims_Analyst",
            agent_variable_name="agents",
            history_variable_name="history",
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[claims_analyst],
            function=termination_function,
            kernel=kernel,
            result_parser=lambda r: "complete" in str(r.value[0]).lower(),
            history_variable_name="history",
            maximum_iterations=20,
        ),
    )

    return group_chat

## Processing a Claim
This function executes the group chat workflow, starting with a user message (the claim scenario). 

In [48]:
async def process_claim(group_chat: AgentGroupChat, claim_id: str):
    print(f"🔄 Starting analysis for claim {claim_id}")
    
    # Reset the chat completion state
    group_chat.is_complete = False
    
    # Add the initial message
    await group_chat.add_chat_message(
        ChatMessageContent(
            role=AuthorRole.USER, 
            content=f"Please analyze claim {claim_id} from the provided documents. Each agent should follow their specific instructions and respond in turn."
        )
    )

    current_agent = None
    message_count = 0
    
    try:
        async for response in group_chat.invoke():
            if response.name != current_agent:
                current_agent = response.name
                print(f"\n👤 {current_agent}:")
                message_count += 1
            
            if response.content:
                print(response.content.strip())

            # Prevent premature termination
            if message_count < 4:
                group_chat.is_complete = False

        if message_count < 4:
            print("\n⚠️ Warning: Not all agents completed their analysis")
        else:
            print("\n✅ Claim analysis complete!")
            
    except Exception as e:
        print(f"\n❌ Error during analysis: {str(e)}")

## Run the Entire Workflow

In this final cell, we:

- Initialize kernel & documents
- Create agents
- Setup group chat
- Process the claim

Run this cell and review the outputs. You can add additional Markdown cells afterwards to comment on the results.

In [49]:
kernel = create_kernel()
claims_analyst, policy_verifier, risk_assessor = asyncio.run(create_agents(kernel, document_files))
group_chat = asyncio.run(setup_group_chat(kernel, claims_analyst, policy_verifier, risk_assessor))

asyncio.run(process_claim(group_chat, "HURTX-2024-0456"))

# Cleanup
asyncio.run(claims_analyst.delete())


🔄 Starting analysis for claim HURTX-2024-0456

👤 Claims_Analyst:
###InitialAnalysisofClaimHURTX-2024-0456####ClaimDetails:-**ClaimID:**HURTX-2024-0456-**Claimant:**ContosoElectronicsInc.(Store#TX-230)-**Incident:**HurricaneAlicia-**DateofIncident:**May15,2024-**DamageReported:**-Severeroofdamageandwaterintrusion-Floodedinventorystoragewithsignificantlossofelectronics-Electricalpanelsdamaged-Structuralconcernsnoted####EstimatedDamageCosts:-**InventoryLoss:**$380,000-**StructuralDamage:**$200,000-**EquipmentReplacement:**$160,000-**BusinessInterruption:**TBD####PolicyCoverage:-**TotalInsuredValue:**$2,000,000-**HurricaneCoverage:**Included-**FloodCoverage:**Included(Deductible:$50,000)-**BusinessInterruptionCoverage:**Includesasub-limitof$500,000####WeatherReportVerification:-HurricaneAliciamadelandfallintheHoustonareaasaCategory3storm,withmaximumsustainedwindsofaround120mphandtotalrainfallof10-14inches,confirminghighdamagepotential【4:1†source】.####MissingInformation:-**ProfessionalStruc

True